In [ ]:
'''
    This notebook generates conditional proabability distributions for given datasets.
'''

In [ ]:
import math
import numpy as np
import pickle

from utils.encode_datasets import read_and_encode_dataset
from utils.util_functions import list_to_string

In [ ]:
def get_final_alphabet_list(alphabet_list, base_string = "", output_list = []):

    if len(alphabet_list) == 1:
        for i in alphabet_list[0]:
            if base_string == "":
                output_list.append(str(i))
            else:
                output_list.append(base_string + " " + str(i))
        return output_list
    new_output_list = output_list
    new_base_string = base_string

    for i in alphabet_list[0]:
        if base_string == "":
            new_base_string = str(i)
        else:
            new_base_string = base_string + " " + str(i)
        new_output_list = get_final_alphabet_list(alphabet_list[1:], base_string = new_base_string, output_list = new_output_list)
        
    return new_output_list

def get_ordered_alphabet_(attributes):
    alphabet_list = []

    if not(isinstance(attributes, list)):
        attributes = [attributes]

    for i in attributes:
        alphabet_list.append(get_ordered_alphabet_individual(i))
    return (get_final_alphabet_list(alphabet_list=alphabet_list, base_string = "", output_list = []))
        
def get_ordered_alphabet_individual(attribute):
    return (np.unique(original_data[attribute].values))

def ignore_nan(arr):
    return max(filter(lambda x: not (math.isinf(x) or math.isnan(x)), arr))

def get_CMF(filtered_original_data, filtered_perturb_data, Z_alphabet, X_k_alphabet, samples):
    probability_matrix = np.ones((len(X_k_alphabet), len(Z_alphabet)))
    len_of_original_dataset = np.shape(filtered_original_data)[0]

    index_x_k = 0
    index_z = 0
    
    for index_i, i in enumerate(filtered_perturb_data[:samples,:]):
        index_z = Z_alphabet.index(list_to_string(i)[:-1])
        index_x_k = X_k_alphabet.index(list_to_string(filtered_original_data[index_i%len_of_original_dataset])[:-1])
        probability_matrix[index_x_k][index_z] += 1

    for i in range(np.shape(probability_matrix)[0]):
        for j in range(np.shape(probability_matrix)[1]):
            if probability_matrix[i][j] < 3:
                probability_matrix[i][j] = 0

    probability_matrix /= np.sum(probability_matrix)
    conditional_matrix = np.transpose(np.transpose(probability_matrix)/np.sum(probability_matrix, axis=1))

    return conditional_matrix


In [ ]:
datasets = ["celeba"] # ['adult', 'cardio', 'celeba', 'SPM_2016', 'dss'] #SPM-2023

file_location = '/datasets/celeba.txt'

def validate(cmf, column):
    for index_i, i in enumerate(column[:]):
        for index_j, j in enumerate(column):
            if i == j:
                continue

            key_ = str(i) + ' ' + str(j)
                
            if key_ not in cmf.keys():
                print("Incomplete CMF")
                raise ValueError(f"Error key {index_i}, {index_j}")
                
def read_csv(name):
    original_data, COLUMNS = read_and_encode_dataset(dataset_name=name)
    return original_data

for dataset in datasets:
    try:
        with open(f'{file_location}/{dataset}/{dataset}_cmf.pkl', 'rb') as file:
            CMF_dict, COLUMNS = pickle.load(file)
            validate(CMF_dict, COLUMNS)
    except:
        original_data = read_csv(dataset)

        COLUMNS = original_data.columns
        try:
            with open(f'{file_location}/{dataset}/{dataset}_cmf.pkl', 'rb') as file:
                CMF_dict, _ = pickle.load(file)
        except:
            CMF_dict = {}
        
        for index_i, i in enumerate(COLUMNS[:]):
            for index_j, j in enumerate(COLUMNS):

                if i == j:
                    continue

                X_k = [i]
                Z = [j]
                
                X_k_alphabet = get_ordered_alphabet_(X_k)
                Z_alphabet = get_ordered_alphabet_(Z)

                filtered_original_data = original_data[X_k].values
                filtered_perturb_data = original_data[Z].values

                key_ = str(i) + ' ' + str(j)
                print("key ", index_i, index_j)
                if key_ not in CMF_dict.keys():
                    CMF = get_CMF(filtered_original_data, filtered_perturb_data, Z_alphabet, X_k_alphabet, samples=original_data.shape[0])
                    CMF_dict[key_] = CMF

            with open(f'{file_location}/{dataset}_cmf.pkl', 'wb') as file:
                pickle.dump((CMF_dict, COLUMNS), file)